# Visualización en Pytorch usando Tensorboard

TensorBoard es una de las herramientas más potentes de visualización en machine learning. Permite realizar funciones interesantes como:

- Visualizar métricas como la pérdida y la exactitud.
- Visualizar los grafos creados por el modelo.
- Ver histógramas de los parámetros y como cambian en el tiempo.
- Ver imágenes y datos de de audio o texto.

PyTorch permite trabajar con TensorBoard. En la versión PyTorch 1.1.0, TensorBoard se soportaba de manera experimental pero desde la versión PyTorch 1.2.0 ya está integrada plenamente.

Primero, tendremos que instalar la extensión de Tensorboard para notebooks y cargarla, como se ve a continuación.

In [ ]:
# pip install -q tf-nightly
# Carga la extension
%load_ext tensorboard

Después, crearemos el directorio donde guardaremos los logs que almacene TensorBoard.

In [3]:
import os
logs_base_dir = "runs"
os.makedirs(logs_base_dir, exist_ok=True)

Creamos una clase con varias capas de redes neuronales, instanciamos un objeto de esta clase y lo optimizamos con autograd.

Para usar TensorBoard, primero creamos un objeto SummaryWriter() que permite crear un archivo de eventos en el directorio indicado (por defecto sería “runs”) y añadir eventos en el archivo.

Después, le indicamos qué eventos guardar en el archivo creado. En este caso, vamos a registrar dos tipos de eventos o logs:

- El grafo creado por el modelo, lo que hará con writer.add_graph(net,input).
- La pérdida en cada iteración, que la registraremos con writer.add_scalar(‘training loss’,loss.data).

In [4]:
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from torch.autograd import Variable

In [12]:
input_size= 10
output_size= 1   
hidden_size= 50   
train_size= 5000
batch_size=10    
num_epochs= 10     
learning_rate =1e-3 

In [13]:

writer = SummaryWriter()

len=int(train_size/batch_size)

input=torch.normal(0, 1, size=(train_size, input_size))
input=input/input.max()

linear1=nn.Linear(input_size, output_size)

with torch.no_grad():
  label=linear1(input)

In [14]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    
        self.fc1 = nn.Linear(input_size, hidden_size)  
        self.relu1 = nn.ReLU()                          
        self.fc2 = nn.Linear(hidden_size, output_size) 
    
    def forward(self, x):                              
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [15]:

net = Net(input_size, hidden_size, output_size)

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for i in range(len):      
        optimizer.zero_grad()
        input1=input[i*batch_size:(i+1)*batch_size]
        label1=label[i*batch_size:(i+1)*batch_size]                             
        output = net(input1)                             
        loss = criterion(output, label1)                 
        loss.backward()                                   
        optimizer.step()                                  
        writer.add_scalar('training loss',loss.data)                              

In [16]:
writer.add_graph(net,input)
writer.close()

Ahora llamaremos a Tensorboard.

In [17]:
%tensorboard --logdir {logs_base_dir}

Reusing TensorBoard on port 6006 (pid 9496), started 0:03:13 ago. (Use '!kill 9496' to kill it.)

Por último, borraremos el directorio creado para almacenar el archivo de logs.

In [ ]:
# !rm -rf runs